In [1]:
import pandas as pd 
import numpy as np
from datetime import datetime
from tqdm import tqdm
import pickle

In [2]:
import config
from scripts.utils import setup_web3, get_logs, save_raw_logs, load_csv_data
from scripts.fetch_logs import *
from scripts.process_validator_delegator import *
from scripts.process_user_rewards_vault import *

/Users/benoitrobaglia/miniforge3/envs/beraboost/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
w3 = setup_web3()

In [4]:
last_block = w3.eth.get_block("latest")["number"]
print(last_block)

2505226


In [5]:
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [6]:
fetch_validator_delegator_logs(w3, 1, last_block, timestamp)

Fetching validator-delegator logs from block 1 to 2500176...
Found 40901 validator-delegator logs
Saved 40901 raw logs to raw_logs/validator_delegator/logs_2025-03-18_10:51:59.pkl


## Load raw logs

In [7]:
file_name = 'logs_2025-03-18_10:51:59.pkl'

In [8]:
raw_logs = pickle.load(open(f'raw_logs/validator_delegator/{file_name}', 'rb'))

In [9]:
decoded_logs = decode_all_validator_delegator_logs_multiprocessing(raw_logs)

Decoding 40901 logs using 11 processes...


Decoding logs: 100%|██████████| 40901/40901 [00:02<00:00, 16914.34it/s]


In [10]:
decoded_logs.to_csv(f'processed_data/validator_delegator/decoded_logs/{file_name}.csv', index=False)

In [11]:
validator_delegator_state = decoded_logs.groupby(['validator_address', 'delegator_address']).amount_bgt_delegated.sum().reset_index()

In [35]:
validator_delegator_state.to_csv(f"processed_data/validator_delegator/states/validator_delegator_state_{timestamp.replace(' ', '_')}.csv", index=False)

## User RV data

In [13]:
fetch_user_rewards_vault_logs(w3, 1, last_block, timestamp)

Fetching user-rewards vault logs from block 1 to 2500176...
Querying logs for 5 reward vaults: 0xC2BaA8443cDA8EBE51a640905A8E6bc4e1f9872c, 0x086f82fa0cA310Cc835a9DB4f53697687ef149c7, 0x17376aD6167a5592FbEAA42e6068c132474a513d, 0x6649Bc987a7c0fB0199c523de1b1b330cd0457A8, 0xF99be47baf0c22B7eB5EAC42c8D91b9942Dc7e84
Found 336901 user-rewards vault logs across all reward vaults
Saved 336901 raw logs to raw_logs/user_rewards_vault/logs_2025-03-18 10:51:59.pkl


In [17]:
raw_logs_urv = pickle.load(open(f'raw_logs/user_rewards_vault/{file_name}', 'rb'))

In [18]:
raw_logs_urv[-1]

AttributeDict({'address': '0x17376aD6167a5592FbEAA42e6068c132474a513d',
 'topics': [HexBytes('0x9e71bc8eea02a63969f509818f2dafb9254532904319f9dbda79b67bd34a5f3d'),
  HexBytes('0x0000000000000000000000000df14916796854d899576cbde69a35bafb923c22')],
 'data': HexBytes('0x0000000000000000000000000000000000000000000000000001c78c602f170d'),
 'blockHash': HexBytes('0xe038d63e0e6b979d4afc9e09ca99588f70dd971c838613d7d0b224d02eaa5ecb'),
 'blockNumber': 2500173,
 'blockTimestamp': '0x67d94236',
 'transactionHash': HexBytes('0xa9342f2ce452b771550e909ce05e9ffbba0446815ab6cb60de949b90be1320b2'),
 'transactionIndex': 30,
 'logIndex': 90,
 'removed': False})

In [19]:
urv_df = decode_all_user_rewards_vault_logs_multiprocessing(raw_logs_urv)

Decoding 336901 logs using 11 processes...


Decoding logs: 100%|██████████| 336901/336901 [00:13<00:00, 25810.45it/s]


In [20]:
urv_df

,tx_hash,block_number,user_address,rv_address,amount,event_type
0,0x1f6dc5bc86cd55d53e009ed6ce08ce918243d80e01b0...,479405,0x160d0e134b78bf083bd7f03b5d9fcbcb1c6ff27a,0xC2BaA8443cDA8EBE51a640905A8E6bc4e1f9872c,8.977546e-01,0x9e71bc8eea02a63969f509818f2dafb9254532904319...
1,0x5748c324a5471d8c396e197584417d81ccf711713f11...,778512,0x59945c5be54ff1d8deb0e8bc7f132f950da910a2,0xF99be47baf0c22B7eB5EAC42c8D91b9942Dc7e84,4.199976e+07,0x9e71bc8eea02a63969f509818f2dafb9254532904319...
2,0x5748c324a5471d8c396e197584417d81ccf711713f11...,778512,0xd8c53e0e7cf3ecfe642a03a30ec30681ef4159a9,0x6649Bc987a7c0fB0199c523de1b1b330cd0457A8,2.699778e+07,0x9e71bc8eea02a63969f509818f2dafb9254532904319...
3,0x14d1c20b813d8969755cc566634af2d423be78f7d8be...,778557,0xcb4d5e7026c3f86348c80d918ba9e511295399f1,0xC2BaA8443cDA8EBE51a640905A8E6bc4e1f9872c,1.294314e+00,0x9e71bc8eea02a63969f509818f2dafb9254532904319...
4,0x3728e4bde621920d7dc387dc80910fe77ead803b3f37...,778563,0xacb0e88f7497bfc5fe2d139008792d9ba81a0f6e,0x086f82fa0cA310Cc835a9DB4f53697687ef149c7,2.870550e-02,0x9e71bc8eea02a63969f509818f2dafb9254532904319...
...,...,...,...,...,...,...
336896,0x24f34d31dcdf66ac345dde2870ac51539526f626ac3f...,2500157,0x1419515d3703d8f2cc72fa6a341685e4f8e7e8e1,0xF99be47baf0c22B7eB5EAC42c8D91b9942Dc7e84,9.941341e-02,0x9e71bc8eea02a63969f509818f2dafb9254532904319...
336897,0xda8ba8b9b99f3838799fd5cf506e86c293fa3767ed24...,2500168,0xe2d8941dfb85435419d90397b09d18024ebeef2c,0xC2BaA8443cDA8EBE51a640905A8E6bc4e1f9872c,2.916203e-02,0x9e71bc8eea02a63969f509818f2dafb9254532904319...
336898,0xca892e4adb8a9deb23160e4766c6d092c2b4f5c345ce...,2500170,0xbbb228b0d7d83f86e23a5ef3b1007d0100581613,0x6649Bc987a7c0fB0199c523de1b1b330cd0457A8,4.500986e+00,0x9e71bc8eea02a63969f509818f2dafb9254532904319...
336899,0xe6b4a51f4b6879be4fb719e5a977d7576fd0dca80496...,2500171,0x78beda3a06443f51718d746ade95b5fac094633e,0x086f82fa0cA310Cc835a9DB4f53697687ef149c7,6.799220e-04,0x9e71bc8eea02a63969f509818f2dafb9254532904319...


In [25]:
urv_df.to_csv(f"processed_data/user_rewards_vault/decoded_logs/{file_name.split('.')[0]}_decoded.csv", index=False)

In [26]:
urv_state = urv_df.groupby(['user_address', 'rv_address']).amount.sum().reset_index()

In [27]:
timestamp

'2025-03-18 10:51:59'

In [29]:
urv_state.loc[urv_state.amount > 0].reset_index(drop=True).to_csv(f"processed_data/user_rewards_vault/states/user_rewards_vault_state_{timestamp.replace(' ', '_')}.csv", index=False)

## Beraboost v1

In [36]:
vd_state = pd.read_csv(f"processed_data/validator_delegator/states/validator_delegator_state_{timestamp.replace(' ', '_')}.csv")
urv_state = pd.read_csv(f"processed_data/user_rewards_vault/states/user_rewards_vault_state_{timestamp.replace(' ', '_')}.csv")

In [37]:
c1_pubkey = '0xb6a44182b44298daee8e3773048b978dc36b19c71a47f90318ea5d5309fc06e9'

In [136]:
def run_beraboost_v1(val_pubkey, user_rv_state, vd_state):
    # Compute the total staked amount and user share
    total_staked = user_rv_state.groupby('rv_address').amount.sum()
    user_rv_state['user_share'] = user_rv_state.apply(
        lambda x: x.amount / total_staked[x.rv_address], axis=1)

    # Filter active delegators of the val_pubkey
    delegators_val = vd_state.loc[vd_state.validator_address == val_pubkey].reset_index(
        drop=True)
    delegators_val = delegators_val.loc[delegators_val.delegator_address.isin(
        user_rv_state.user_address)].reset_index(drop=True)
    delegators_val['amount_bgt_delegated_norm'] = delegators_val.amount_bgt_delegated / \
        delegators_val.amount_bgt_delegated.sum()

    # Compute the user share of the active delegators
    delegator_rv = user_rv_state.loc[user_rv_state.user_address.isin(
        delegators_val.delegator_address)].reset_index(drop=True)
    delegator_rv_matrix = delegator_rv.pivot(
        index='user_address', columns='rv_address', values='user_share').fillna(0)

    # Get max user share for each delegator
    max_liq = np.argmax(delegator_rv_matrix, axis=1)

    # Compute the weight
    w = pd.DataFrame({'delegator_address': delegator_rv_matrix.index,
                      'rv_address': delegator_rv_matrix.columns[max_liq]}).merge(delegators_val[['delegator_address', 'amount_bgt_delegated_norm']],
                                                                                 on='delegator_address',
                                                                                 how='left')

    cb = w.groupby('rv_address').amount_bgt_delegated_norm.sum()
    cb = round(cb*1e4)  # Converting values in BP and rounding
    cb = cb.loc[cb > 0]  # Filtering non zero values
    cb_formatted = list(cb.items())
    return cb_formatted


In [137]:
run_beraboost_v1(c1_pubkey, urv_state, vd_state)

[('0x086f82fa0cA310Cc835a9DB4f53697687ef149c7', 2481.0),
 ('0x6649Bc987a7c0fB0199c523de1b1b330cd0457A8', 28.0),
 ('0xC2BaA8443cDA8EBE51a640905A8E6bc4e1f9872c', 2095.0),
 ('0xF99be47baf0c22B7eB5EAC42c8D91b9942Dc7e84', 5396.0)]

## Berachef weight updates

In [6]:
fetch_berachef_weight_update_logs(w3, 1, last_block, timestamp)

Fetching BeraChef weight update logs from block 1 to 2505226...
Found 139 BeraChef weight update logs
Saved 139 raw logs to raw_logs/berachef_weight_updates/logs_2025-03-18_13:33:58.pkl


In [7]:
timestamp

'2025-03-18 13:33:58'

In [9]:
raw_logs_berachef = pickle.load(open(f"{config.RAW_LOGS_DIR['berachef_weight_updates']}/logs_{timestamp.replace(' ', '_')}.pkl", 'rb'))

In [12]:
l = raw_logs_berachef[-1]
l

AttributeDict({'address': '0xdf960E8F3F19C481dDE769edEDD439ea1a63426a',
 'topics': [HexBytes('0x09fed3850dff4fef07a5284847da937f94021882ecab1c143fcacd69e5451bd8'),
  HexBytes('0x001e7ef887da57571a396802eb0eef14014ea14a2a1543e55509f2d1c237a6f6')],
 'data': HexBytes('0x0000000000000000000000000000000000000000000000000000000000260e9200000000000000000000000000000000000000000000000000000000000000400000000000000000000000000000000000000000000000000000000000000005000000000000000000000000086f82fa0ca310cc835a9db4f53697687ef149c700000000000000000000000000000000000000000000000000000000000009c300000000000000000000000017376ad6167a5592fbeaa42e6068c132474a513d00000000000000000000000000000000000000000000000000000000000009c3000000000000000000000000c2baa8443cda8ebe51a640905a8e6bc4e1f9872c0000000000000000000000000000000000000000000000000000000000000db0000000000000000000000000f99be47baf0c22b7eb5eac42c8d91b9942dc7e8400000000000000000000000000000000000000000000000000000000000002ed0000000000000000000000006649

In [ ]:
def decode_weight_update_log(w3, log):
    event_signature = '0x' + log['topics'][0].hex() if isinstance(log['topics'][0], bytes) else log['topics'][0]
    assert event_signature == config.BERACHEF.event_signatures['ActivateRewardAllocation'], f"Invalid event signature: {event_signature}"
    # Load contract
    abi = get_contract_abi(config.BERACHEF.abi_url)
    berachef_contract = w3.eth.contract(address=config.BERACHEF.address, abi=abi)
    decoded_log = berachef_contract.events.ActivateRewardAllocation().process_log(log)
        
    return {
        "block_number": log['blockNumber'],
        "transaction_hash": '0x' + log['transactionHash'].hex() if isinstance(log['transactionHash'], bytes) else log['transactionHash'],
        "validator_address": '0x' + log['topics'][1].hex() if isinstance(log['topics'][1], bytes) else log['topics'][1],
        "start_block": decoded_log.args.startBlock,
        "weights": [(w.receiver, w.percentageNumerator) for w in decoded_log.args.weights]
    }


In [20]:
abi = get_contract_abi(config.BERACHEF.abi_url)

In [23]:
berachef_contract = w3.eth.contract(address=config.BERACHEF.address, abi=abi)

In [25]:
pl = berachef_contract.events.ActivateRewardAllocation().process_log(l)

In [31]:
[(w.receiver, w.percentageNumerator) for w in pl.args.weights]

[('0x086f82fa0cA310Cc835a9DB4f53697687ef149c7', 2499),
 ('0x17376aD6167a5592FbEAA42e6068c132474a513d', 2499),
 ('0xC2BaA8443cDA8EBE51a640905A8E6bc4e1f9872c', 3504),
 ('0xF99be47baf0c22B7eB5EAC42c8D91b9942Dc7e84', 749),
 ('0x6649Bc987a7c0fB0199c523de1b1b330cd0457A8', 749)]